In [1]:

from src.data.dataset_reader import DatasetReader
from pathlib import Path
%load_ext autoreload
%autoreload 2
# ignore warnings
import warnings
warnings.filterwarnings('ignore')
data_folder = Path("../data/raw").resolve()

amazon = DatasetReader(data_folder).get_data_csv('Amazon Sale Report.csv')
#int_sales = DatasetReader(data_folder).get_data_csv('International sale Report.csv')
#p_l = DatasetReader(data_folder).get_data_csv('P LMarch2021.csv')
#may_2022 = DatasetReader(data_folder).get_data_csv('May-2022.csv')
#sale_report = DatasetReader(data_folder).get_data_csv('Sale Report.csv')


This dataset provides an in-depth look at the profitability of e-commerce sales. It contains data on a variety of sales channels, including Shiprocket and INCREFF, as well as financial information on related expenses and profits. The columns contain data such as SKU codes, design numbers, stock levels, product categories, sizes and colors. In addition to this we have included the MRPs across multiple stores like Ajio MRP , Amazon MRP , Amazon FBA MRP , Flipkart MRP , Limeroad MRP Myntra MRP and PaytmMRP along with other key parameters like amount paid by customer for the purchase , rate per piece for every individual transaction Also we have added transactional parameters like Date of sale months category fulfilledby B2b Status Qty Currency Gross amt . This is a must-have dataset for anyone trying to uncover the profitability of e-commerce sales in today's marketplace.
This dataset provides a comprehensive overview of e-commerce sales data from different channels covering a variety of products. Using this dataset, retailers and digital marketers can measure the performance of their campaigns more accurately and efficiently.

The following steps help users make the most out of this dataset:

Analyze the general sales trends by examining info such as month, category, currency, stock level, and customer for each sale. This will give you an idea about how your e-commerce business is performing in each channel.

Review the Shiprocket and INCREF data to compare and analyze profitability via different fulfilment methods. This comparison would enable you to make better decisions towards maximizing profit while minimizing costs associated with each method’s referral fees and fulfillment rates.

Compare prices between various channels such as Amazon FBA MRP, Myntra MRP, Ajio MRP etc using  the corresponding columns for each store (Amazon MRP etc). You can judge which stores are offering more profitable margins without compromising on quality by analyzing these pricing points in combination with other information related to product sales (TP1/TP2 - cost per piece).

Look at customer specific data such as TP 1/TP 2 combination wise Gross Amount or Rate info in terms price per piece or total gross amount generated by any SKU dispersed over multiple customers with relevant dates associated to track individual item performance relative to others within its category over time periods shortlisted/filtered appropriately.. Have an eye on items commonly utilized against offers or promotional discounts offered hence crafting strategies towards inventory optimization leading up-selling operations.?

Finally Use Overall ‘Stock’ details along all the P & L Data including Yearly Expenses_IIGF information record for takeaways which might be aimed towards essential cost cutting measures like switching amongst delivery options carefully chosen out of Shiprocket & INCREFF leadings away from manual inspections catering savings under support personnel outsourcing structures.?  

By employing a comprehensive understanding on how our internal subsidiaries perform globally unless attached respective audits may provide us remarkably lower operational costs servicing confidence; costing far lesser than being incurred taking into account entire pallet shipments tracking sheets representing current level supply chains efficiencies achieved internally., then one may finally scale profits exponentially increases cut down unseen losses followed up introducing newer marketing campaigns necessarily tailored according playing around multiple goods based spectrums due powerful backing suitable transportation boundaries set carefully

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
amazon.head(3)
#amazon.query('SKU == "SET389-KR-NP-S"').head(1)
                                             


,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,Size,ASIN,Courier Status,Qty,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,S,B09KXVBD7Z,NaN,0,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,NaN
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,3XL,B09K3WFS32,Shipped,1,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,XL,B07WV4JV4D,Shipped,1,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,NaN


In [3]:
# Create a new feature called sales that is the  the count of "Amount" grouped by "SKU" in the amazon dataset
#amazon['sales'] = amazon.groupby('SKU')['Amount'].transform('count')
#amazon.drop(columns=[ 'Order ID', 'ASIN', 'Unnamed: 22', 'index'], inplace=True
amazon['Date'] = pd.to_datetime(amazon['Date'])
amazon['ship-postal-code'] = amazon['ship-postal-code'].astype('object')
amazon_beta =  amazon[['Order ID','Date', 'SKU', 'Qty', 'Amount']]

In [15]:
amazon_beta =  amazon[['Order ID','Date', 'SKU', 'Qty', 'Amount']]

In [4]:
from src.data.eda import Eda
eda = Eda(    
)

eda.missing_values_table(amazon_beta)

The selected dataframe has 4 columns and 1 columns with missing values.


,Missing Values,% of Total Values
Amount,7795,6.0


In [21]:
amazon_beta.rename(columns={'Amount': 'price'}, inplace=True)

In [16]:
amazon_beta.head()
# Create a column called "sales" that is the count of the "Order ID" grouped by "SKU" and "Date" in the amazon_beta dataset
amazon_beta['sales'] = amazon_beta.groupby(['SKU', 'Date'])['Order ID'].transform('count')


In [17]:
amazon_beta.query('SKU == "SET389-KR-NP-S"').sort_values(by=['Date'], ascending=True)

,Order ID,Date,SKU,Qty,Amount,sales
47784,406-4493775-5535509,2022-04-01,SET389-KR-NP-S,1,749.00,1
41605,408-5859715-9071508,2022-04-05,SET389-KR-NP-S,1,0.00,1
36996,402-3547090-0699503,2022-04-08,SET389-KR-NP-S,1,680.00,1
18468,405-2959500-2916317,2022-04-19,SET389-KR-NP-S,1,680.00,1
16932,406-8338408-6778764,2022-04-20,SET389-KR-NP-S,0,647.62,1
10192,408-7468849-2134719,2022-04-24,SET389-KR-NP-S,1,680.00,1
5555,408-9553867-3715507,2022-04-27,SET389-KR-NP-S,1,680.00,1
0,405-8078784-5731545,2022-04-30,SET389-KR-NP-S,0,647.62,1
90657,405-2042325-2489944,2022-05-01,SET389-KR-NP-S,1,680.00,2
89452,407-2478454-8055511,2022-05-01,SET389-KR-NP-S,1,680.00,2


In [8]:
# show duplicates in amazon_beta 
#amazon_beta[amazon_beta.duplicated(keep=False)].sort_values(by='SKU', ascending=True).head(10)
# drop duplicates in amazon_beta
#amazon_beta.drop_duplicates(inplace=True)

In [20]:
eda.missing_values_table(amazon_beta)

The selected dataframe has 6 columns and 1 columns with missing values.


,Missing Values,% of Total Values
Amount,7795,6.0


In [23]:
amazon_beta.sort_values(by=['SKU', 'Date'], ascending=True, inplace=True)

In [76]:
from src.data.feature_engineering import FeatureEngineeringProcess
fep = FeatureEngineeringProcess()

In [25]:
amazon_beta_fe1 = fep.price_sales_correlation_features_updated(amazon_beta, 15, [1, 2], [1,2],)
amazon_beta_fe1.head(3)

,Order ID,Date,SKU,Qty,price,sales,f_corr_1_1,f_corr_1_2,f_corr_2_1,f_corr_2_2
38234,403-7294691-2015536,2022-04-07,AN201-RED-M,1,229.0,1,NaN,NaN,NaN,NaN
90899,404-8297237-4618714,2022-05-01,AN201-RED-M,1,229.0,1,NaN,NaN,NaN,NaN
96990,404-9268921-7038734,2022-06-24,AN201-RED-XL,1,301.0,1,NaN,NaN,NaN,NaN


In [27]:
#print this instruction: amazon_beta_fe1.query('SKU == "SET389-KR-NP-S"') without index
amazon_beta_fe1.query('SKU == "SET389-KR-NP-S"').head(3)



,Order ID,Date,SKU,Qty,price,sales,f_corr_1_1,f_corr_1_2,f_corr_2_1,f_corr_2_2
47784,406-4493775-5535509,2022-04-01,SET389-KR-NP-S,1,749.0,1,NaN,NaN,NaN,NaN
41605,408-5859715-9071508,2022-04-05,SET389-KR-NP-S,1,0.0,1,NaN,NaN,NaN,NaN
36996,402-3547090-0699503,2022-04-08,SET389-KR-NP-S,1,680.0,1,NaN,NaN,NaN,NaN


In [33]:
amazon_beta_fe2 = fep.normalize_features(amazon_beta_fe1, 8, 15)

In [34]:
amazon_beta_fe2.query('SKU == "SET389-KR-NP-S"').T

,47784,41605,36996,18468,16932,10192,5555,0,89452,90657,81612,79604,76788,74119,72887,73227,69377,68184,63150,61715,55684,52271,52394,50956,122224,120210,120220,118560,116026,113528,107922,108247,106201,96481,96006,94760
Order ID,406-4493775-5535509,408-5859715-9071508,402-3547090-0699503,405-2959500-2916317,406-8338408-6778764,408-7468849-2134719,408-9553867-3715507,405-8078784-5731545,407-2478454-8055511,405-2042325-2489944,405-6543898-7204304,408-9372786-1186724,406-5709592-5938750,408-4324865-1447532,406-6229189-6697906,408-5443635-9313910,406-6995612-9636346,402-8338057-9605163,408-8571787-7192312,407-6133617-4202742,408-6727965-6193141,403-3152840-0384305,407-9551962-0653910,402-2536827-2784338,171-0288010-2242709,406-3668757-2000361,404-4771127-5991532,405-5277435-6412332,403-6801316-1649157,402-2526028-2309155,402-7546538-1614720,408-1152101-2203536,408-5236007-1292358,404-3034160-6680315,402-4666860-7106725,171-0243036-7059512
Date,2022-04-01 00:00:00,2022-04-05 00:00:00,2022-04-08 00:00:00,2022-04-19 00:00:00,2022-04-20 00:00:00,2022-04-24 00:00:00,2022-04-27 00:00:00,2022-04-30 00:00:00,2022-05-01 00:00:00,2022-05-01 00:00:00,2022-05-05 00:00:00,2022-05-07 00:00:00,2022-05-09 00:00:00,2022-05-11 00:00:00,2022-05-12 00:00:00,2022-05-12 00:00:00,2022-05-15 00:00:00,2022-05-16 00:00:00,2022-05-20 00:00:00,2022-05-21 00:00:00,2022-05-26 00:00:00,2022-05-29 00:00:00,2022-05-29 00:00:00,2022-05-30 00:00:00,2022-06-05 00:00:00,2022-06-06 00:00:00,2022-06-06 00:00:00,2022-06-08 00:00:00,2022-06-09 00:00:00,2022-06-11 00:00:00,2022-06-15 00:00:00,2022-06-15 00:00:00,2022-06-16 00:00:00,2022-06-24 00:00:00,2022-06-25 00:00:00,2022-06-26 00:00:00
SKU,SET389-KR-NP-S,SET389-KR-NP-S,SET389-KR-NP-S,SET389-KR-NP-S,SET389-KR-NP-S,SET389-KR-NP-S,SET389-KR-NP-S,SET389-KR-NP-S,SET389-KR-NP-S,SET389-KR-NP-S,SET389-KR-NP-S,SET389-KR-NP-S,SET389-KR-NP-S,SET389-KR-NP-S,SET389-KR-NP-S,SET389-KR-NP-S,SET389-KR-NP-S,SET389-KR-NP-S,SET389-KR-NP-S,SET389-KR-NP-S,SET389-KR-NP-S,SET389-KR-NP-S,SET389-KR-NP-S,SET389-KR-NP-S,SET389-KR-NP-S,SET389-KR-NP-S,SET389-KR-NP-S,SET389-KR-NP-S,SET389-KR-NP-S,SET389-KR-NP-S,SET389-KR-NP-S,SET389-KR-NP-S,SET389-KR-NP-S,SET389-KR-NP-S,SET389-KR-NP-S,SET389-KR-NP-S
Qty,1,1,1,1,0,1,1,0,1,1,1,1,1,1,1,1,0,1,1,1,0,1,0,0,1,1,1,1,1,0,1,1,1,1,1,1
price,749.0,0.0,680.0,680.0,647.62,680.0,680.0,647.62,680.0,680.0,648.0,629.0,648.0,648.0,648.0,648.0,617.14,648.0,648.0,648.0,NaN,648.0,617.14,617.14,648.0,648.0,648.0,648.0,648.0,617.14,648.0,648.0,648.0,648.0,648.0,648.0
sales,1,1,1,1,1,1,1,1,2,2,1,1,1,1,2,2,1,1,1,1,1,2,2,1,1,2,2,1,1,1,2,2,1,1,1,1
f_corr_1_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.067493,0.067753,0.066807,0.066853,0.0669,0.066899,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.066697
f_corr_1_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.068525,0.068961,0.066963,0.06706,0.067158,0.067157,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.066725
f_corr_2_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.067509,0.067836,0.066944,0.067038,0.067134,0.067132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.066727
f_corr_2_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.068563,0.069135,0.067251,0.06745,0.067652,0.06765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.066782


In [77]:
amazon_beta_fe3 = fep.filter_stability_periods(amazon_beta_fe2, 7, 0.04)

In [72]:
# print without index amazon_beta_fe3.query('SKU == "SET389-KR-NP-S"') without index column


amazon_beta_fe3[['Order ID', 'Date', 'SKU', 'price', 'sales','avg_price_last_7_days', 'price_variation']].query('SKU == "SET389-KR-NP-S"')


# NOTE: Still seeing repeated values in the "price" column. Either low the threshold or increase removal_percentage


,Order ID,Date,SKU,price,sales,avg_price_last_7_days,price_variation
47784,406-4493775-5535509,2022-04-01,SET389-KR-NP-S,749.00,1,NaN,0.000000
41605,408-5859715-9071508,2022-04-05,SET389-KR-NP-S,0.00,1,NaN,0.000000
36996,402-3547090-0699503,2022-04-08,SET389-KR-NP-S,680.00,1,NaN,0.000000
18468,405-2959500-2916317,2022-04-19,SET389-KR-NP-S,680.00,1,NaN,0.000000
16932,406-8338408-6778764,2022-04-20,SET389-KR-NP-S,647.62,1,NaN,0.000000
10192,408-7468849-2134719,2022-04-24,SET389-KR-NP-S,680.00,1,NaN,0.000000
5555,408-9553867-3715507,2022-04-27,SET389-KR-NP-S,680.00,1,588.088571,0.156288
0,405-8078784-5731545,2022-04-30,SET389-KR-NP-S,647.62,1,573.605714,0.129033
79604,408-9372786-1186724,2022-05-07,SET389-KR-NP-S,629.00,1,663.517143,0.052021
55684,408-6727965-6193141,2022-05-26,SET389-KR-NP-S,NaN,1,NaN,NaN
